In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np


In [2]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

# data = data.drop(
#     columns=["release_date", "key", "loudness",
#              "explicit", "popularity", "duration_ms", "danceability", "energy", "speechiness",
#              "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

# data = data.drop(
#     columns=["release_date", "name"])

data.head(5)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, glam rock, mandopop, album r..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[folk, protopunk, rock, regional mexican, glam..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."


In [3]:
unique_favourite_genres = set()
for genres in data['favourite_genres']:
  for genre in genres:
    unique_favourite_genres.add(genre)

unique_genres = set()
for genres in data['genres']:
  for genre in genres:
    unique_genres.add(genre)

# show all common genres between all favourite_genres and genres
# TODO chyba zle zliczam genres, skoro w analize wyszlo 2875 gatunkow
print("unique_favourite_genres", len(unique_favourite_genres))
print("unique_genres", len(unique_genres))
common_genres = unique_favourite_genres.intersection(unique_genres)
print("common genres", common_genres)

# remove genres that are not in favourite_genres
# data['genres'] = data['genres'].apply(
#     lambda x: [genre for genre in x if genre in unique_favourite_genres])

data.head()

# TODO remove ??? moze niepotrzebne

unique_favourite_genres 46
unique_genres 1766
common genres {'new wave', 'permanent wave', 'latin alternative', 'country rock', 'alternative rock', 'brill building pop', 'soft rock', 'metal', 'latin pop', 'lounge', 'turkish pop', 'latin rock', 'psychedelic rock', 'folk', 'adult standards', 'rock en espanol', 'new romantic', 'c-pop', 'dance pop', 'ranchera', 'hard rock', 'filmi', 'latin', 'hoerspiel', 'europop', 'new wave pop', 'quiet storm', 'pop rock', 'alternative metal', 'soul', 'motown', 'rock', 'mellow gold', 'funk', 'mandopop', 'mpb', 'pop', 'album rock', 'vocal jazz', 'singer-songwriter', 'tropical', 'regional mexican', 'classic rock', 'j-pop', 'art rock', 'blues rock'}


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, glam rock, mandopop, album r..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[folk, protopunk, rock, regional mexican, glam..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."


In [4]:
# balance set

skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_data.head()

skipped_data (3621, 34)
not_skipped_data (6260, 34)


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-03-23 18:05:18.096,147,0kVB6PeqBbN2HhwJdWumeZ,732,The Wall Street Shuffle,40,236373,0,6i6WlGzQtXtz7GcC5H5st5,1974,...,True,10cc,"[album rock, art rock, classic rock, classic u...",True,1,3,3,18,"[album rock, art rock, classic rock, classic u...","[rock, classic uk pop, glam rock, hoerspiel, a..."
1,2023-01-22 05:30:03.002,141,4pJKzul9oXW1lBQmOpf96m,646,Deep Dark Dungeon,38,129133,0,06nsZ3qSOYZ2hPVIMcr1IN,1981-02,...,False,J.J. Cale,"[album rock, blues, blues rock, classic rock, ...",False,1,1,6,5,"[album rock, blues, blues rock, classic rock, ...","[rock, album rock, country rock, folk rock, cl..."
2,2023-02-23 05:54:20.102,125,7t6zMZSO6DfCeE4IcinxUJ,443,Chala Chala Navbala,0,193320,0,61JrslREXq98hurYL2hYoc,1943-12-31,...,False,Lata Mangeshkar,"[classic bollywood, desi pop, filmi, sufi]",False,0,2,3,5,"[classic bollywood, desi pop, filmi, sufi, cla...","[regional mexican, classic bollywood, sufi, cl..."
3,2023-02-26 04:48:49.355,149,62UVjAjo8ra1PVSeHfnxk3,754,I Love Paris,21,353200,0,3XOVABzceOUTbR3iEz0ImO,1959-01-01,...,False,Cal Tjader,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...",True,0,2,6,4,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...","[new wave pop, latin jazz, jazz, lounge, bossa..."
4,2023-01-31 09:21:51.897,134,7DDfGy1cstvJGETyYYnfig,559,The One to Sing the Blues,30,187213,0,1DFr97A9HnbV3SKTJFu62M,1991-02-26,...,True,Motörhead,"[album rock, alternative metal, hard rock, met...",True,0,1,1,9,"[album rock, alternative metal, hard rock, met...","[rock, album rock, brill building pop, metal, ..."


In [5]:
def normalize(data):
  return (data - np.min(data)) / (np.max(data) - np.min(data))

# # Combine genres and favourite_genres
all_genres = list(balanced_data['favourite_genres'] + balanced_data['genres'])

# # One-hot encode the genres
mlb = MultiLabelBinarizer()
mlb.fit(all_genres)

# encoded_all_genres = mlb.fit_transform(all_genres)

# TODO moze trzeba inaczej enkodowac?
#encoded_genres_with_favourite_genres = mlb.fit_transform(balanced_data['genres_with_favourite_genres'])
# encoded_common_genres = mlb.fit_transform(balanced_data['common_genres'])

# TODO enkodowac genres_with_favourite_genres naraz

# Split encoded_genres into genres and favourite_genres
# encoded_favourite_genres = encoded_all_genres[:, :len(balanced_data['favourite_genres'][0])]
# encoded_genres = encoded_all_genres[:, len(balanced_data['favourite_genres'][0]):]

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])
# encoded_common_genres = mlb.transform(data['common_genres'])

timestamp_normalized = balanced_data['timestamp'].values.reshape(-1, 1)
timestamp_normalized = normalize(timestamp_normalized)

popularity_normalized = balanced_data['popularity'].values.reshape(-1, 1)
popularity_normalized = normalize(popularity_normalized)

tempo_normalized = balanced_data['tempo'].values.reshape(-1, 1)
tempo_normalized = normalize(tempo_normalized)

duration_ms_normalized = balanced_data['duration_ms'].values.reshape(-1, 1)
duration_ms_normalized = normalize(duration_ms_normalized)

loudness_normalized = balanced_data['loudness'].values.reshape(-1, 1)
loudness_normalized = normalize(loudness_normalized)

danceability_normalized = balanced_data['danceability'].values.reshape(-1, 1)
danceability_normalized = normalize(danceability_normalized)

energy_normalized = balanced_data['energy'].values.reshape(-1, 1)
energy_normalized = normalize(energy_normalized)

number_of_matching_genres_normalized = balanced_data['number_of_matching_genres'].values.reshape(-1, 1)
number_of_matching_genres_normalized = normalize(number_of_matching_genres_normalized)

hour_of_day_normalized = balanced_data['hour_of_day'].values.reshape(-1, 1)
hour_of_day_normalized = normalize(hour_of_day_normalized)

day_of_week_normalized = balanced_data['day_of_week'].values.reshape(-1, 1)
day_of_week_normalized = normalize(day_of_week_normalized)

premium_user_normalized = balanced_data['premium_user'].astype(int).values.reshape(-1, 1)
premium_user_normalized = normalize(premium_user_normalized)

# create data frame from data genres, data favourite_genres, encoded genres, encoded favourite_genres
df = pd.DataFrame(
    data={
      'genres': balanced_data['genres'],
      'favourite_genres': balanced_data['favourite_genres'],
      'skipped': balanced_data['skipped']})

df.head(50)

,genres,favourite_genres,skipped
0,"[album rock, art rock, classic rock, classic u...","[soft rock, hoerspiel, ranchera]",True
1,"[album rock, blues, blues rock, classic rock, ...","[tropical, permanent wave, mellow gold]",False
2,"[classic bollywood, desi pop, filmi, sufi]","[classic rock, regional mexican, pop rock]",False
3,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...","[psychedelic rock, new wave pop, alternative r...",True
4,"[album rock, alternative metal, hard rock, met...","[hoerspiel, quiet storm, brill building pop]",True
5,"[kentucky hip hop, pop, pop rap, r&b, rap, trap]","[hoerspiel, j-pop, filmi]",False
6,"[album rock, art rock, classic rock, folk rock...","[soft rock, hoerspiel, ranchera]",True
7,"[brooklyn indie, indie pop]","[c-pop, latin pop, ranchera]",True
8,"[album rock, canadian singer-songwriter, class...","[filmi, regional mexican, folk]",False
9,"[contemporary country, country, country dawn, ...","[hoerspiel, regional mexican, folk]",False


In [6]:
# Concatenate the one-hot encoded columns
X = np.concatenate([encoded_genres, encoded_favourite_genres], axis=1)
# X = np.concatenate([encoded_track_name], axis=1)
# X = np.concatenate(
#     [popularity_normalized, duration_ms_normalized, danceability_normalized, energy_normalized,
#      number_of_matching_genres_normalized], axis=1)

#
# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_train == 0))
print("Y_train skipped", np.count_nonzero(y_train == 1))
print("Y_train skipped %", np.count_nonzero(y_train == 1) / len(y_train))

Y_train not skipped 2872
Y_train skipped 2921
Y_train skipped % 0.5042292421888486


In [7]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
print("TEST")
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))

print("TRAIN")
y_pred = rf_model.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred))
print("Classification report:\n", classification_report(y_train, y_pred))


TEST
Accuracy: 0.6238785369220152
Confusion matrix:
 [[451 298]
 [247 453]]
Classification report:
               precision    recall  f1-score   support

           0       0.65      0.60      0.62       749
           1       0.60      0.65      0.62       700

    accuracy                           0.62      1449
   macro avg       0.62      0.62      0.62      1449
weighted avg       0.63      0.62      0.62      1449

TRAIN
Accuracy: 0.9249093733816676
Confusion matrix:
 [[2630  242]
 [ 193 2728]]
Classification report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92      2872
           1       0.92      0.93      0.93      2921

    accuracy                           0.92      5793
   macro avg       0.93      0.92      0.92      5793
weighted avg       0.93      0.92      0.92      5793

